In [1]:
import preproc_tabular
from custom_dataset import getDataset
import neural_net
import torch
from torch import nn


target = [1,2,1,4,2,4,3,5,1,0] #TARGET FAKE
img_path = 'D:\\ORS\\Data\\ResizedImages\\'
tab_path = 'D:\\ORS\\Data\\prova.xlsx'

data, references, descriptions, len_desc = preproc_tabular.get_tabular('D:\\ORS\\Data\\prova.xlsx')
newdata, newdescription, newreferences = preproc_tabular.duplicate_row(img_path, data, descriptions, references)
train, val, test = getDataset(newreferences, newdata, newdescription, "ciao")
 
modello = neural_net.create_model(len_desc, len(data.columns))

learning_rate = 1e-3
batch_size = 1
epochs = 2
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(modello.parameters(), lr=learning_rate) 
early_stop = 3

stable_loss = 0
prec_loss = 20000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    neural_net.train_loop(train, modello, loss_fn, optimizer, 1)
    val_loss = neural_net.validation_loop(val, modello, loss_fn)
    stop, stable_loss = neural_net.early_stopping(val_loss, prec_loss, stable_loss, early_stop)
    if stop:
        break
print("Done!")

d:\Programmi\Python3.12\Lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
d:\Programmi\Python3.12\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Epoch 1
-------------------------------


d:\Programmi\Python3.12\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 1.000000
loss: 1.000000
loss: 16.000000
loss: 0.000000
loss: 9.000000
loss: 25.000000
Validation Error: 
 Avg loss: 3.000000 

Epoch 2
-------------------------------
loss: 1.000000
loss: 1.000000
loss: 16.000000
loss: 0.000000
loss: 9.000000
loss: 25.000000
Validation Error: 
 Avg loss: 3.000000 

Done!
